In [ ]:
import { env } from './utils.ts';

In [3]:
import { ChatOpenAI } from '@langchain/openai';
import { ChatPromptTemplate, MessagesPlaceholder } from '@langchain/core/prompts';
import { RunnableSequence, RunnablePassthrough } from "@langchain/core/runnables";
import { StringOutputParser } from "@langchain/core/output_parsers";

const summaryModel = new ChatOpenAI({
  configuration: {
    baseURL: env["QWEN_BASE_URL"],
    apiKey: env["QWEN_API_KEY"],
  },
  model: env["QWEN_MODEL"],
});
const summaryPrompt = ChatPromptTemplate.fromTemplate(`
Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary

Current summary:
{summary}

New lines of conversation:
{new_lines}

New summary:
`);

const summaryChain = RunnableSequence.from([
  summaryPrompt,
  summaryModel,
  new StringOutputParser()
] as any);

In [ ]:
import { ChatMessageHistory } from 'langchain/stores/message/in_memory';
import { getBufferString } from 'langchain/memory'

const chatModel = new ChatOpenAI({
  configuration: {
    baseURL: env["QWEN_BASE_URL"],
    apiKey: env["QWEN_API_KEY"],
  },
  model: env["QWEN_MODEL"],
});

const chatPrompt = ChatPromptTemplate.fromMessages([
  ['system', `You are a helpful assistant. Answer all questions to the best of your ability.
    
    Here is the chat history summary:
    {history_summary}
  `],
  ['human', '{input}']
]);

let summary = '';
const chatHistory = new ChatMessageHistory();

const chatChain = RunnableSequence.from([
  {
    input: new RunnablePassthrough({
      func: (input) => {
        chatHistory.addUserMessage(input)
      }
    }),
  },
  RunnablePassthrough.assign({
    history_summary: () => summary
  }),
  chatPrompt,
  chatModel,
  new StringOutputParser(), 
  new RunnablePassthrough({
    func: async (input) => {
      chatHistory.addAIMessage(input)
      const messages = await chatHistory.getMessages();
      const new_lines = getBufferString(messages)
      const newSummary = await summaryChain.invoke({
        summary,
        new_lines
      });
      chatHistory.clear();
      summary = newSummary;
    }
  })
] as any)


In [5]:
await chatChain.invoke('我现在饿了')

new_lines:  Human: 我现在饿了
AI: 那你可以吃点东西啊。你附近有什么好吃的吗？或者你想要什么类型的美食？我可以给你一些建议哦。
summary:  你告诉AI你现在饿了，AI建议你可以吃点东西，并询问你附近有什么好吃的或你想要什么类型的美食，以便给你提供一些建议。


"那你可以吃点东西啊。你附近有什么好吃的吗？或者你想要什么类型的美食？我可以给你一些建议哦。"

In [6]:
await chatChain.invoke('我今天想吃方便面')

new_lines:  Human: 我今天想吃方便面
AI: 那你可以选择一款你喜欢口味的方便面，煮着吃或者泡着吃都可以。如果你想要更丰富一点，可以加一个鸡蛋、一些蔬菜或者火腿进去，这样更有营养哦。你平时喜欢吃什么口味的方便面呢？
summary:  你告诉AI你现在饿了，AI建议你可以吃点东西，并询问你附近有什么好吃的或你想要什么类型的美食，以便给你提供一些建议。你告诉AI你今天想吃方便面，AI建议你可以选择一款喜欢的口味，煮泡皆可，并建议添加鸡蛋、蔬菜或火腿让营养更丰富，同时询问你平时喜欢什么口味的方便面。


"那你可以选择一款你喜欢口味的方便面，煮着吃或者泡着吃都可以。如果你想要更丰富一点，可以加一个鸡蛋、一些蔬菜或者火腿进去，这样更有营养哦。你平时喜欢吃什么口味的方便面呢？"